In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time
from datetime import datetime
import FinanceDataReader as fdr
import os

### 기본배경
1. JR 글로벌리츠 5일 평균 글 2개정도 생산되나? 최근 금리 급등으로 리츠주가 반짝 관심을 받아 초단기 상승을 하니 글이 되게 많아짐
2. 토론이 많은 곳에 단기 투자자가 많다는 생각을 했는데 이 생각이 현실성 있는건지 생각해봤음.
   1. 내가 생각하는 단기투자자란? => 장 시작하자마자 구매 , 일정 금액 이상 목표치 또는 목표 %를 설정하고 장 마감전까지 팔지 못하면 손절하는 투자행태
   2. 따라서 08:00-16:00까지 게시글 리젠에 집중해서 봐야함.
   3. 단기투자가 발생하면? 주가 요동, 거래량 급증
   4. 그럼 내가 찾고 싶은 포인트는 네이버 증권 토론글 증가가 주가 요동또는 거래량 급증과 연관성이 있다는건가?
   5. 거래량, 주가 요동 기준은 5일 선으로 하면 될 것 같다. 5일 선과 당일 주가, 거래량 변동을 점검 하면 되지 않을까?
   6. T-Test는 표본끼리 비교하는 거 아닌가? 근데 당일 주가는 개별 원소이니까 비교하면 안되지 않나?
3. 단기 투자자와 토론글 증가가 관련없다면 아니라면 토론이 증가한 이유는 뭘까? 어떤 시장 요인과 연관성이 있을까? 외인 거래량? 가격 상승? 골든 크로스? 시장호재?


### Exploring analysis
1. 실험기간은 작년 4월 ~ 올해 4월
2. 자료수집 기업은..? 200개 기업 전부를 조사하는게 의미가 있으려나? 
3. 개인 참여자의 관심 지표 => 해당 종목 하루 게시글 개수, 장 시간 동안(09:00-16:00) 시간당 평균 글 리젠 개수, 장 시간 외 시간당 평균 글 리젠 개수
6. T-Test로 가설이 맞는지 검증할 수 있으려나. 

### 향후 계획
1. 번외 JP 모건은 정말 단타가 많을까? jp모건 거래량을 수집할 수 있으려나..?
2. 토론에서 가장 많이 쓰인 단어는 뭘까? 하락?

In [ ]:
codes_df = pd.read_excel('data/codes.xlsx') # 삼성증권에서 조건식에 맞는 종목 정보 저장하였음. 관련 내용 불러오기
codes_df['종목코드'] = [format(code,'06') for code in codes_df['종목코드']] # 종목코드 6자리로 변경



### 네이버 댓글 날짜 및 작성자 scrap하기

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36"
}

for code, name in zip(codes_df["종목코드"], codes_df["종목명"]):
    print(f"종목코드 : {code} || 종목명 : {name}")  # scraping 중인 code 및 name 출력
    b = 0
    while True:  ### 원하는 날짜가 21-03-01 이후 댓글이므로 21-03-01이 몇페이지 부터 시작하는지 찾는 식
        time.sleep(0.5)  # 과부화 방지
        b += 1
        u = b * 10  # 10page씩 건너띄기
        url = f"https://finance.naver.com/item/board.naver?code={code}&page={u}"

        result = requests.get(url, headers=headers)
        soup = BeautifulSoup(result.text, "html.parser")  # 페이지 불러와서 bs4로 변경

        k = soup.find("span", "tah p10 gray03")  # 페이지 별 최신 날짜 찾기
        
        print(
            b, "", u, "", datetime.strptime(k.get_text(), "%Y.%m.%d %H:%M")
        )  # 진행과정 출력


        if datetime.strptime("2021.03.01 00:00", "%Y.%m.%d %H:%M") > datetime.strptime(
            k.get_text(), "%Y.%m.%d %H:%M"
        ):
            break  # 21-03-01 이전 날짜 식별되면 찾기 중단

    list_date = []  # 날짜
    list_id = []  # Id
    for i in range(1, u + 1):  # u : scrap 해야하는 페이지 수
        if i % 5 == 0:
            time.sleep(0.11)  # 5페이지마다 쉬어주기
        url = f"https://finance.naver.com/item/board.naver?code={code}&page={i}"
        result = requests.get(url, headers=headers)
        soup = BeautifulSoup(result.text, "html.parser")  # 페이지 불러와서 bs4로 변경
        a = soup.find_all("span", "tah p10 gray03")
        data = [i.get_text() for i in a][0::2]  # 시간만 추출하기
        list_date.extend(data)  # 순서대로 시간 정렬

        b = soup.find_all("td", "p11")
        i_d = [re.sub("\s", "", i.get_text()) for i in b][0:-1]
        list_id.extend(i_d)  # id 정렬

    print(len(list_date), len(list_id), list_date[-1])  # 수집한 정보크기 추력
    total = pd.DataFrame([list_date, list_id]).T  # row로 정리된 자료 column으로 변경
    total.to_csv(f"data/{name}_posts.csv")  # 자료 저장



### 주식 Price 찾기

In [ ]:
import os
import re
import pandas as pd
import time


ist_dir = os.listdir("./data")  # data folder file을 list로 변환
codes = [
    re.sub("_posts.csv", "", a) for a in ist_dir if "_posts.csv" in a
]  # _posts.csv 가 들어간 list 만들기
code_raw = pd.read_excel("data/codes.xlsx")  # raw 주식정보 불러오기
code_raw["종목코드"] = [format(a, "06") for a in code_raw["종목코드"]]  # 주식코드 6자리로 변경
code_last = code_raw[code_raw["종목명"].isin(codes)]  # isin이 value로 row 찾기에 너무 좋다.

for code, name in zip(code_last["종목코드"], code_last["종목명"]):
    df = fdr.DataReader(
        code, "2020-10-01", "2022-04-25"
    )  # 20,60일 선을 만들어야하므로 충분히 긴 기간으로 설정
    df["20"] = df["Close"].rolling(20).mean()  # 20일 평균선
    df["60"] = df["Close"].rolling(60).mean()  # 60일 평균선
    df.to_csv(f"data/{name}_price.csv")  # 자료 저장
    time.sleep(0.1)  # 과부화 방지
